In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from pandas_datareader import data as pdr
from datetime import datetime, timedelta
import data_util  
yf.pdr_override()
from sklearn.svm import SVC

sns.set(rc={'figure.figsize':(20, 15)})

temp = dict(layout=go.Layout(font=dict(family="Franklin Gothic", size=12), width=800))

import warnings
warnings.filterwarnings('ignore')


In [3]:
apple_stocks=data_util.stock_data('AAPL', 4,'Apple')

[*********************100%%**********************]  1 of 1 completed


In [4]:
from prophet import Prophet
from prophet.plot import plot_plotly

# Assuming you have already imported necessary libraries and loaded the data

# Reset index of apple_stocks DataFrame
apple_stocks.reset_index(inplace=True)
# apple_stocks.set_index('Date',inplace=True)
apple_stocks_temp = apple_stocks[['Date', 'Close']].rename(columns={'Date': 'ds', 'Close': 'y'})

apple_stocks_temp_test = apple_stocks_temp.tail(100)

apple_stocks_temp_train = apple_stocks_temp.head(len(apple_stocks_temp) - 100)

FBprophet = Prophet(daily_seasonality=True)

FBprophet.fit(apple_stocks_temp_train)

future = FBprophet.make_future_dataframe(periods=365)

forecast = FBprophet.predict(future)

fig = plot_plotly(FBprophet, forecast)

fig.add_trace(go.Scatter(x=apple_stocks_temp_test['ds'], y=apple_stocks_temp_test['y'], mode='markers', name='Test Data'))

fig.show()


23:33:45 - cmdstanpy - INFO - Chain [1] start processing
23:33:45 - cmdstanpy - INFO - Chain [1] done processing


In [5]:
forecast['abs_error'] = np.abs(forecast['yhat'] - apple_stocks_temp_test['y'])

# Calculate mean error per day
mean_error_per_day = forecast['abs_error'].mean()

print("Mean error per day:", mean_error_per_day," USD")

Mean error per day: 16.070947043036803  USD
